# Importing packages

In [7]:
import os

import numpy as np
import pandas as pd

from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as img
plt.style.use('fivethirtyeight')

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix, accuracy_score

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter

from torchvision.transforms import Compose, ToTensor, Resize, RandomResizedCrop, Normalize
from torchvision.datasets import ImageFolder

from data_preparation.arrange_train_images import arrange_images
from data_preparation.create_train_csv import create_csv

# Data/Image exploration

Set the value of arange_folder to True in the cell below if you want to arrange the content of the train folder into cat and dog folders.

In [2]:
arange_folder = True

if arange_folder:
    arrange_images()
else:
    pass

0it [00:00, ?it/s]


Let's create a csv file for train data by setting get_csv to True. If get_csv is False we will skip this step.

In [4]:
get_csv = True

if get_csv:
    create_csv()
else:
    pass

Let's check the train.csv file.

In [5]:
train_df = pd.read_csv('train.csv')
train_df.head()

,file,class
0,0.jpg,cat
1,1.jpg,cat
2,10.jpg,cat
3,100.jpg,cat
4,1000.jpg,cat


Creating a target column and showing class distribution

In [6]:
map_dict = {"cat": 1, "dog": 0}

train_df['target'] = train_df['class'].map(map_dict)

train_df['class'].value_counts(normalize=True)

class
cat    0.5
dog    0.5
Name: proportion, dtype: float64

The two classes are equally distributed in the train data. So we can use accuracy as a metric for gauging performance of future models. Let's also create fold column which will indicate to which fold a file in a row belongs to. Moreover we need to sustain the same proportion between the classes in each fold.

In [8]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for fold, (train_idx, val_idx) in enumerate(skf.split(train_df, train_df['target'])):
    train_df.loc[val_idx, 'fold'] = int(fold + 1)

train_df.head()

,file,class,target,fold
0,0.jpg,cat,1,1.0
1,1.jpg,cat,1,5.0
2,10.jpg,cat,1,4.0
3,100.jpg,cat,1,2.0
4,1000.jpg,cat,1,5.0
